In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import warnings
import lightgbm
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [ ]:
gbm = lightgbm.Booster(model_file='../input/trial5/try5.txt')

In [ ]:
def sortandfill(df):
    df = df.ffill()
    return df

def transformData(datafilled):
#     datafilled2 = datafilled[datafilled['DateCode'].notna()]
    
    LE_TypeOfDocument = preprocessing.LabelEncoder()
    datafilled['TypeOfDocument'] = LE_TypeOfDocument.fit_transform(datafilled['TypeOfDocument'])
    LE_TypeOfCurrentPeriod = preprocessing.LabelEncoder()
    datafilled['TypeOfCurrentPeriod'] = LE_TypeOfCurrentPeriod.fit_transform(datafilled['TypeOfCurrentPeriod'])
    datafilled.fillna(0,inplace=True)
    datafilled.set_index("Date", inplace=True)
    try:
        datafilled.replace("－",0, inplace=True)
    except:
        pass
    return datafilled

In [ ]:
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

counter=0
for (stock_prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    financials = financials[financials['SecuritiesCode'].notna()]
    financials['SecuritiesCode'] = financials['SecuritiesCode'].astype(np.int64)
    data = pd.merge(stock_prices, financials, left_on=['SecuritiesCode', 'Date'], right_on=['SecuritiesCode', "Date"], how="left")
    datafilled = data.groupby("SecuritiesCode").apply(sortandfill)
    datafilled = transformData(datafilled)
    dataCols = [i for i in datafilled.columns]
    deletCols = ['RowId','DateCode','DisclosedDate', 'DisclosedTime','CurrentPeriodEndDate','TypeOfCurrentPeriod','CurrentFiscalYearStartDate','CurrentFiscalYearEndDate']
    features = [dataCols.remove(i) for i in deletCols]
    X = datafilled[dataCols]
    X = X.astype(np.float64)
    sample_prediction['Target'] = gbm.predict(X)
    sample_prediction = sample_prediction.sort_values(by = "Target", ascending = False)
    sample_prediction['Rank'] = np.arange(len(sample_prediction.index))
    sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending = True)
    sample_prediction.drop(["Target"], axis = 1, inplace=True)
    submission = sample_prediction[["Date", "SecuritiesCode", "Rank"]]  
    env.predict(submission)
    counter += 1